# 18650 Battery Models

## Aryan, Avery, Cormac, & Tyler

In [4]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
#import torch
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm

In [5]:
regular = pd.read_csv('.\\battery_alt_dataset\\regular_alt_batteries.csv')

C:\Users\tyler\AppData\Local\Temp\ipykernel_14540\1882604773.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  regular = pd.read_csv('.\\battery_alt_dataset\\regular_alt_batteries.csv')


In [17]:
def battLabeller(df):
    '''
    df - battery dataset
    feed battery dataset, add column to denote discharge phase
    '''
    label = 1
    df['dischargePhase'] = 0
    # for time in range(len(df['start_time'])-1):
    for time in tqdm(range(len(df['start_time'])-1), total=len(df['start_time'])-1):
        if df['mode'][time] == -1:
            df.at[time, 'dischargePhase'] = label
            if df['mode'][time+1] == 0 or df['mode'][time+1] == 1:
                label += 1
    return df

In [32]:
regular = regular[regular['mode'] != 0]
regular = regular[regular['mode'] != 1]

In [40]:
regular

,start_time,time,mode,voltage_charger,temperature_battery,voltage_load,current_load,temperature_mosfet,temperature_resistor,mission_type,dischargePhase,startTime,finishTime
324,2022-07-19 11:10:00,300.523,-1.0,8.340,22.963,-0.026,0.152000,22.64,23.20,0.0,1.0,0,0
325,2022-07-19 11:10:00,301.463,-1.0,8.041,23.155,8.329,2.523000,22.65,23.21,0.0,1.0,0,0
326,2022-07-19 11:10:00,302.404,-1.0,8.030,23.17,8.320,2.521000,22.66,23.20,0.0,1.0,0,0
327,2022-07-19 11:10:00,303.345,-1.0,8.023,23.186,8.312,2.523000,22.68,23.21,0.0,1.0,0,0
328,2022-07-19 11:10:00,304.288,-1.0,8.016,23.201,8.305,2.521000,22.71,23.21,0.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28357171,2022-10-16 12:55:00,1506502.662,-1.0,6.361,41.868,6.356,14.816191,31.22,27.72,1.0,4896.0,0,0
28357172,2022-10-16 12:55:00,1506503.726,-1.0,6.392,42.073,6.409,14.850649,31.24,27.77,1.0,4896.0,0,0
28357173,2022-10-16 12:55:00,1506504.785,-1.0,6.365,42.259,6.379,14.848449,31.28,27.81,1.0,4896.0,0,0
28357174,2022-10-16 12:55:00,1506505.841,-1.0,6.382,42.437,6.377,14.819122,31.31,27.86,1.0,4896.0,0,0


In [16]:
def timeFinder(df):
    '''
    df - dataframe
    takes dataframe and finds time from start of phase to end of phase
    '''
    df['startTime'] = 0
    df['finishTime'] = 0
    # for phase in df['dischargePhase'].unique():
    for phase in tqdm(df['dischargePhase'].unique(), total=df['dischargePhase'].unique()):
        currPhase = df[df['dischargePhase'] == phase]
        
        # Find start and end time
        print(phase)
        start = currPhase['time'].iloc[0] 
        finish = currPhase['time'].iloc[-1]
        
        # Update start and end time
        df.loc[df['dischargePhase'] == phase, 'startTime'] = start
        df.loc[df['dischargePhase'] == phase, 'finishTime'] = finish

In [46]:
regular[regular['dischargePhase'] == 4]

,start_time,time,mode,voltage_charger,temperature_battery,voltage_load,current_load,temperature_mosfet,temperature_resistor,mission_type,dischargePhase,startTime,finishTime
18434,2022-07-22 10:42:00,17623.174,-1.0,8.383,27.009,-0.026,0.550303,19.54,22.57,1.0,4.0,17623.174,18162.392
18435,2022-07-22 10:42:00,17624.251,-1.0,8.379,27.179,8.694,0.593648,19.54,22.56,1.0,4.0,17623.174,18162.392
18436,2022-07-22 10:42:00,17625.242,-1.0,7.448,27.203,7.455,16.291283,19.53,22.60,1.0,4.0,17623.174,18162.392
18437,2022-07-22 10:42:00,17626.230,-1.0,7.392,27.416,7.395,16.309729,19.61,22.61,1.0,4.0,17623.174,18162.392
18438,2022-07-22 10:42:00,17627.224,-1.0,7.350,27.76,7.361,16.300513,19.71,22.63,1.0,4.0,17623.174,18162.392
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18972,2022-07-22 10:42:00,18158.108,-1.0,5.168,96.902,5.125,16.237060,29.29,31.98,1.0,4.0,17623.174,18162.392
18973,2022-07-22 10:42:00,18159.148,-1.0,5.129,96.971,5.069,16.195385,29.25,31.97,1.0,4.0,17623.174,18162.392
18974,2022-07-22 10:42:00,18160.177,-1.0,5.079,97.022,5.031,16.204028,29.19,31.93,1.0,4.0,17623.174,18162.392
18975,2022-07-22 10:42:00,18161.355,-1.0,5.021,97.022,4.970,16.154918,29.13,31.92,1.0,4.0,17623.174,18162.392


In [52]:
cols = ['voltage_charger','temperature_battery','current_load','dischargePhase','startTime','finishTime']
regular = regular[cols]
regular = regular.dropna(subset=['temperature_battery'])

In [6]:
def process_battery_data(df):
    '''
    Takes in df of battery data and reduces each phase
    to a single row, averaging temp, voltage, and current
    and finding the time to death for each discharge phase
    
    Parameters:
    df (dataFrame): input df of battery data sent through
    battLabeller and timeFinder
    
    Returns:
    df (transformed): averaged values and new column
    end_time which is the difference of finishTime and startTime
    '''
    # Get time_end via difference
    df['time_end'] = (df['finishTime'] - df['startTime'])
    
    # Cast columns to floats
    numeric_cols = ['voltage_charger','temperature_battery','current_load']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col],errors = 'coerce')
    
    # Group by dischargePhase and aggregate other columns
    aggregated_df = df.groupby('dischargePhase').agg({
        'voltage_charger':'mean',
        'temperature_battery':'mean',
        'current_load':'mean',
        'time_end':'first'
    }).reset_index()
    
    return aggregated_df

In [61]:
regular = process_battery_data(regular)

In [62]:
regular

,dischargePhase,voltage_charger,temperature_battery,current_load,time_end
0,1.0,6.993291,25.916404,2.519990,3503.232
1,2.0,6.448755,64.757509,16.253802,538.454
2,3.0,6.474374,67.185977,16.246017,540.349
3,4.0,6.472878,67.642913,16.249888,539.218
4,5.0,6.353341,70.845676,17.602098,494.909
...,...,...,...,...,...
4891,4892.0,6.156449,62.168234,15.119306,477.210
4892,4893.0,6.148608,62.504518,15.121327,476.649
4893,4894.0,6.148097,62.199106,15.121059,475.834
4894,4895.0,6.163466,61.660934,15.120537,467.765


In [63]:
regular.to_csv('regular.csv')

In [7]:
#recommissioned = pd.read_csv('.\\battery_alt_dataset\\recommissioned_batteries.csv')
#regular = pd.read_csv('.\\battery_alt_dataset\\regular_alt_batteries.csv')
second = pd.read_csv('.\\battery_alt_dataset\\second_life_batteries.csv')

In [13]:
def process_battery_data(df):
    '''
    Takes in df of battery data and reduces each phase
    to a single row, averaging temp, voltage, and current
    and finding the time to death for each discharge phase
    
    Parameters:
    df (dataFrame): input df of battery data sent through
    battLabeller and timeFinder
    
    Returns:
    df (transformed): averaged values and new column
    end_time which is the difference of finishTime and startTime
    '''
    # Run battery df through functions time_end via difference
    df = battLabeller(df)
    df = df[df['mode'] != 0]
    df = df[df['mode'] != 1]
    df = timeFinder(df)
    cols = ['voltage_charger','temperature_battery','current_load','dischargePhase','startTime','finishTime']
    df = df[cols]
    df = df.dropna(subset=['temperature_battery'])
    
    df['time_end'] = (df['finishTime'] - df['startTime'])
    
    # Cast columns to floats
    numeric_cols = ['voltage_charger','temperature_battery','current_load']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col],errors = 'coerce')
    
    # Group by dischargePhase and aggregate other columns
    aggregated_df = df.groupby('dischargePhase').agg({
        'voltage_charger':'mean',
        'temperature_battery':'mean',
        'current_load':'mean',
        'time_end':'first'
    }).reset_index()
    aggregated_df['mode'] = 1
    
    return aggregated_df

In [14]:
second = process_battery_data(second)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\3167601340.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '600.096' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['dischargePhase'] == phase, 'startTime'] = start
C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\3167601340.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3205.154' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['dischargePhase'] == phase, 'finishTime'] = finish


19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
2

TypeError: 'NoneType' object is not subscriptable

In [ ]:
second.to_csv('second.csv')